In [1]:
from typing import Union

from datasets import Dataset, load_dataset
from peft import LoraConfig  # type: ignore
from pydantic import BaseModel, Field

import xverify as xv

In [2]:
class Tools(BaseModel):
    """
    Run a tool.
    """

    tool_use: xv.XMLToolUse[xv.calculator, xv.search] = Field(
        ..., description="The tool call to use"
    )


class FinalAnswer(BaseModel):
    """
    Return a final answer.
    """

    answer: int = Field(..., description="Final answer to the question")


class Reason_and_Act(BaseModel):
    scratchpad: str = Field(
        ...,
        description="Information from the Observation useful to answer the question",
    )
    reasoning: str = Field(
        ...,
        description="It describes your thoughts about the question you have been asked",
    )
    # response: Tools | FinalAnswer # this still doesn't work
    response: Union[Tools, FinalAnswer]


def tool_response_func(model: Reason_and_Act) -> dict | None:
    return xv.run_tools(model.response)


guided_schema = xv.GuidedSchema(
    Reason_and_Act,
    schema="xml",
    tool_response_func=tool_response_func,
)

In [3]:
def format_prompt(
    prompt: str, system_prompt: str | None = None
) -> list[dict[str, str]]:
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    return messages


def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

max_steps = 10

SYSTEM_PROMPT = f"""\
You are a helpful assistant, responding in XML structured output.

- Think step by step using the scratchpad and reasoning outputs. You have {max_steps - 1} steps to think before responding.
- Use the tools provided. DO NOT rely on your own knowledge when a tool is available to help you.
- Respond with a final answer only once your are absolutely sure you have the answer.

Respond with a XML object, following the schema below:

{guided_schema.doc}
"""

SYSTEM_PROMPT = """\
TEST PROMPT
"""

dataset: Dataset = load_dataset("openai/gsm8k", "main", split="train")  # type: ignore
dataset = dataset.map(lambda x: {
    'prompt': [
        {'role': 'system', 'content': SYSTEM_PROMPT},
        {'role': 'user', 'content': x['question']}
    ],
    'answer': extract_hash_answer(x['answer'])
})

In [ ]:
def exact_answer_reward_func(completions, answer, **_) -> list[float]:
    """Reward function that checks if the final answer matches the expected answer."""

    def _check_answer(trajectory: list[dict[str, str]], answer: int) -> float:
        """Extract the last answer from a trajectory."""
        last_message = trajectory[-1]
        assert last_message["role"] == "assistant", "should be assistant"
        parsed: Reason_and_Act | None = guided_schema.parse(last_message["content"])  # type: ignore
        if parsed is None or not isinstance(parsed.response, FinalAnswer):
            return 0.0
        return 1.0 if parsed.response.answer == answer else 0.0

    return [_check_answer(c, a) for c, a in zip(completions, answer)]


model_name = "Qwen/Qwen2.5-0.5B-Instruct"
model, tokenizer = xv.get_model_and_tokenizer(model_name)


run_name = "gsm8k-calculator-peft_" + model_name.split("/")[-1].lower()
training_args = xv.get_default_grpo_config(run_name, num_gpus=1)

peft_config = LoraConfig(
    r=16,
    lora_alpha=64,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj"],
)


trainer = xv.GRPOGuidedTrainer(
    guided_schema=guided_schema,
    model=model,
    args=training_args,
    train_dataset=dataset,
    peft_config=peft_config,
    reward_funcs=[exact_answer_reward_func],
)
trainer.train()

Using Liger kernel
Applied Liger kernels to Qwen2


You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/home/tom/fun/xverify/.venv/lib/python3.12/site-packages/trl/trainer/grpo_trainer.py:471: UserWarning: The requested device cuda:0 is also being used for training. For higher throughput and to avoid out-of-memory errors, it is recommended to use a dedicated device for vLLM. If this is intentional, you may ignore this warning but should adjust `vllm_gpu_memory_utilization` accordingly.
  warnings.warn(


INFO 03-13 16:06:43 __init__.py:207] Automatically detected platform cuda.
INFO 03-13 16:06:49 config.py:549] This model supports multiple tasks: {'score', 'generate', 'reward', 'classify', 'embed'}. Defaulting to 'generate'.
INFO 03-13 16:06:49 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='Qwen/Qwen2.5-0.5B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-0.5B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda:0, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_e

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 03-13 16:06:50 model_runner.py:1115] Loading model weights took 0.9266 GB
INFO 03-13 16:06:51 worker.py:267] Memory profiling takes 0.83 seconds
INFO 03-13 16:06:51 worker.py:267] the current vLLM instance can use total_gpu_memory (23.58GiB) x gpu_memory_utilization (0.30) = 7.07GiB
INFO 03-13 16:06:51 worker.py:267] model weights take 0.93GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 1.44GiB; the rest of the memory reserved for KV Cache is 4.65GiB.
INFO 03-13 16:06:51 executor_base.py:111] # cuda blocks: 25399, # CPU blocks: 21845
INFO 03-13 16:06:51 executor_base.py:116] Maximum concurrency for 32768 tokens per request: 12.40x
INFO 03-13 16:06:55 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_ut

...

╭──────────────────────────────────────────────────── Step 0 ─────────────────────────────────────────────────────╮
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ │
│ ┃ Prompt                                          ┃ Completion                                       ┃ Reward ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ │
│ │ Boston had .5 feet of snow on the first day of  │ assistant: <Reason_and_Act>                      │   0.00 │ │
│ │ winter.  The next day they got an additional 8  │ <scratchpad>                                     │        │ │
│ │ inches.  Over the next 2 days, 2 inches of the  │                                                  │        │ │
│ │ snow melted.  On the fifth day, they received   │ Let's break down the snowfall situation step by  │        │ │
│ │ another 2 times the amount of snow they         │ step:                                            │        │ │
│ │ received on the first day.  How many feet of    │                                                  │        │ │
│ │ snow do they now have?                          │ 1. On the first day, Boston received 0.5 feet of │        │ │
│ │                                                 │ snow, which converts to 50 inches because there  │        │ │
│ │                                                 │ are 12 inches in a foot.                         │        │ │
│ │                                                 │                                                  │        │ │
│ │                                                 │ 2. On the second day, the snowfall increased by  │        │ │
│ │                                                 │ 8 inches. So, the total snowfall on the second   │        │ │
│ │                                                 │ day is \(50 + 8 = 58\) inches.                   │        │ │
│ │                                                 │                                                  │        │ │
│ │                                                 │ 3. Over the next two days, 2 inches of the snow  │        │ │
│ │                                                 │ melted. So, the total snowfall on the third and  │        │ │
│ │                                                 │ fourth days is \(58 - 2 = 56\) inches.           │        │ │
│ │                                                 │                                                  │        │ │
│ │                                                 │ 4. On the fifth day, they received twice the     │        │ │
│ │                                                 │ amount of snow they received on the first day.   │        │ │
│ │                                                 │ So, the total snowfall on the fifth day is \(2   │        │ │
│ │                                                 │ \times 50 = 100\) inches.                        │        │ │
│ │                                                 │                                                  │        │ │
│ │                                                 │ 5. To add up all the snowfall for the five days, │        │ │
│ │                                                 │ we add the snowfall from each day:               │        │ │
│ │                                                 │    \(50 + 58 + 56 + 100 = 224\) inches.          │        │ │
│ │                                                 │                                                  │        │ │
│ │                                                 │ 6. Since there are 12 inches in a foot, to       │        │ │
│ │                                                 │ convert the total inches of snow to feet, we     │        │ │
│ │                                                 │ divide by 12:                                    │        │ │
│ │                                                 │   

[2025-03-13 16:10:19] INFO lib.rs:450: response: https://lite.duckduckgo.com/lite/ 200
[2025-03-13 16:10:21] INFO lib.rs:450: response: https://lite.duckduckgo.com/lite/ 200
[2025-03-13 16:10:22] INFO lib.rs:450: response: https://lite.duckduckgo.com/lite/ 200


╭──────────────────────────────────────────────────── Step 0 ─────────────────────────────────────────────────────╮
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ │
│ ┃ Prompt                                          ┃ Completion                                       ┃ Reward ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ │
│ │ At Hank’s cafe, he sells big stack pancakes     │ assistant: <Reason_and_Act>                      │   0.00 │ │
│ │ which have 5 pancakes and short stack pancakes  │ <scratchpad>                                     │        │ │
│ │ which have 3 pancakes.  If 9 customers order    │                                                  │        │ │
│ │ the short stack and 6 customers order the big   │ In order to calculate the total number of        │        │ │
│ │ stack, how many pancakes does Hank need to      │ pancakes Hank needs to make, we first need to    │        │ │
│ │ make?                                           │ determine the number of pancakes required for    │        │ │
│ │                                                 │ the short stack and the big stack separately.    │        │ │
│ │                                                 │                                                  │        │ │
│ │                                                 │ - For the short stack, each customer orders 3    │        │ │
│ │                                                 │ pancakes. If 9 customers order this, the total   │        │ │
│ │                                                 │ pancakes for the short stack would be:           │        │ │
│ │                                                 │   3 pancakes × 9 customers = 27 pancakes         │        │ │
│ │                                                 │                                                  │        │ │
│ │                                                 │ - For the big stack, each customer orders 5      │        │ │
│ │                                                 │ pancakes. If 6 customers order this, the total   │        │ │
│ │                                                 │ pancakes for the big stack would be:             │        │ │
│ │                                                 │   5 pancakes × 6 customers = 30 pancakes         │        │ │
│ │                                                 │                                                  │        │ │
│ │                                                 │ - The total number of pancakes needed would be   │        │ │
│ │                                                 │ the sum of the pancakes for the short stack and  │        │ │
│ │                                                 │ the big stack:                                   │        │ │
│ │                                                 │   27 pancakes (from short stack) + 30 pancakes   │        │ │
│ │                                                 │ (from big stack)                                 │        │ │
│ │                                                 │   ------------------------------------           │        │ │
│ │                                                 │   57 pancakes                                    │        │ │
│ │                                                 │                                                  │        │ │
│ │                                                 │ So, Hank needs to make a total of 57 pancakes.   │        │ │
│ │                                                 │                                                  │        │ │
│ │                                                 │ </scratchpad>                                    │        │ │
│ │                                                 │ <reasoning>                                      │        │ │
│ │                                                 │ Ha

Step,Training Loss


╭──────────────────────────────────────────────────── Step 1 ─────────────────────────────────────────────────────╮
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ │
│ ┃ Prompt                                          ┃ Completion                                       ┃ Reward ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ │
│ │ Mary bought a packet of 1500 stickers. She      │ assistant: <Reason_and_Act>                      │   0.00 │ │
│ │ shared them between Susan, Andrew and Sam in    │ <scratchpad>                                     │        │ │
│ │ the ratio 1:1:3 respectively. If Sam gave       │                                                  │        │ │
│ │ Andrew two-thirds of his own share, how many    │ Mary shared 1500 stickers between Susan, Andrew  │        │ │
│ │ stickers does Andrew now have?                  │ and Sam in the ratio 1:1:3 respectively. To find │        │ │
│ │                                                 │ out how much of the stickers each person got, we │        │ │
│ │                                                 │ would multiply the total number of stickers by   │        │ │
│ │                                                 │ the ratio of their share of the total stickers.  │        │ │
│ │                                                 │                                                  │        │ │
│ │                                                 │ The formula would be:                            │        │ │
│ │                                                 │ - Susan gets 1/4 of the total stickers.          │        │ │
│ │                                                 │ - Andrew gets 2/4 of the total stickers.         │        │ │
│ │                                                 │ - Sam gets 3/4 of the total stickers.            │        │ │
│ │                                                 │                                                  │        │ │
│ │                                                 │ Let's calculate the exact amounts for each       │        │ │
│ │                                                 │ person:                                          │        │ │
│ │                                                 │                                                  │        │ │
│ │                                                 │ - Andrew: \( \frac{2}{4} \times 1500 \)          │        │ │
│ │                                                 │ - Susan: \( \frac{1}{4} \times 1500 \)           │        │ │
│ │                                                 │ - Sam: \( \frac{3}{4} \times 1500 \)             │        │ │
│ │                                                 │                                                  │        │ │
│ │                                                 │ Calculating each:                                │        │ │
│ │                                                 │                                                  │        │ │
│ │                                                 │ - Andrew gets: \( \frac{2}{4} \times 1500 =      │        │ │
│ │                                                 │ \frac{1}{2} \times 1500 = 750 \) stickers.       │        │ │
│ │                                                 │ - Susan gets: \( \frac{1}{4} \times 1500 = 375   │        │ │
│ │                                                 │ \) stickers.                                     │        │ │
│ │                                                 │ - Sam gets: \( \frac{3}{4} \times 1500 =         │        │ │
│ │                                                 │ \frac{3}{4} \times 1500 = 1125 \) stickers.      │        │ │
│ │                                                 │                                                  │        │ │
│ │                                                 │ Gi